In [108]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold

### KERAS

#### Character process

In [109]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [110]:
text = pd.read_csv("train.csv", usecols=['text', 'target'])
test_text = pd.read_csv("test.csv", usecols=['text'])

In [111]:
text.drop_duplicates(subset = 'text', keep = False, inplace = True)
text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7434 entries, 0 to 7612
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7434 non-null   object
 1   target  7434 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 174.2+ KB


In [112]:
text['text'] = text['text'].apply(lambda x: x.lower())
test_text['text'] = test_text['text'].apply(lambda x: x.lower())

In [113]:
text.head()

,text,target
0,our deeds are the reason of this #earthquake m...,1
1,forest fire near la ronge sask. canada,1
2,all residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,just got sent this photo from ruby #alaska as ...,1


In [114]:
test_text.head()

,text
0,just happened a terrible car crash
1,"heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,apocalypse lighting. #spokane #wildfires
4,typhoon soudelor kills 28 in china and taiwan


In [115]:
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(text['text'])

In [116]:
# Caracteres muy horrendos
tk.word_index

{'UNK': 1,
 ' ': 2,
 'e': 3,
 't': 4,
 'a': 5,
 'o': 6,
 'i': 7,
 'n': 8,
 's': 9,
 'r': 10,
 'h': 11,
 'l': 12,
 'c': 13,
 'd': 14,
 'u': 15,
 'p': 16,
 'm': 17,
 '/': 18,
 'g': 19,
 'f': 20,
 'y': 21,
 'w': 22,
 '.': 23,
 'b': 24,
 'k': 25,
 'v': 26,
 ':': 27,
 '#': 28,
 'j': 29,
 "'": 30,
 '?': 31,
 'x': 32,
 '@': 33,
 'z': 34,
 '0': 35,
 '1': 36,
 'q': 37,
 '-': 38,
 '2': 39,
 '5': 40,
 '3': 41,
 '4': 42,
 '7': 43,
 '9': 44,
 '6': 45,
 '!': 46,
 '8': 47,
 '\n': 48,
 '_': 49,
 '\x89': 50,
 'û': 51,
 ';': 52,
 '&': 53,
 ')': 54,
 '(': 55,
 '*': 56,
 'ª': 57,
 '|': 58,
 '[': 59,
 ']': 60,
 'å': 61,
 '+': 62,
 'ï': 63,
 'ê': 64,
 '=': 65,
 '÷': 66,
 '%': 67,
 'ò': 68,
 '$': 69,
 '\x9d': 70,
 '~': 71,
 'ó': 72,
 'ì': 73,
 '©': 74,
 '¢': 75,
 '£': 76,
 '^': 77,
 '¨': 78,
 'è': 79,
 '\\': 80,
 '¼': 81,
 '}': 82,
 'ñ': 83,
 '¤': 84,
 '¡': 85,
 '`': 86,
 '{': 87,
 ',': 88,
 'ã': 89,
 'ü': 90,
 'ç': 91,
 'â': 92,
 '«': 93,
 '>': 94,
 '´': 95,
 '¬': 96}

In [117]:
alphabet="abcdefghijklmnñopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$£%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1
    
tk.word_index = char_dict.copy() 
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1 #UNK es el valor mas alto

In [118]:
tk.word_index

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'ñ': 15,
 'o': 16,
 'p': 17,
 'q': 18,
 'r': 19,
 's': 20,
 't': 21,
 'u': 22,
 'v': 23,
 'w': 24,
 'x': 25,
 'y': 26,
 'z': 27,
 '0': 28,
 '1': 29,
 '2': 30,
 '3': 31,
 '4': 32,
 '5': 33,
 '6': 34,
 '7': 35,
 '8': 36,
 '9': 37,
 ',': 38,
 ';': 39,
 '.': 40,
 '!': 41,
 '?': 42,
 ':': 43,
 "'": 44,
 '"': 45,
 '/': 46,
 '\\': 47,
 '|': 48,
 '_': 49,
 '@': 50,
 '#': 51,
 '$': 52,
 '£': 53,
 '%': 54,
 '^': 55,
 '&': 56,
 '*': 57,
 '~': 58,
 '`': 59,
 '+': 60,
 '-': 61,
 '=': 62,
 '<': 63,
 '>': 64,
 '(': 65,
 ')': 66,
 '[': 67,
 ']': 68,
 '{': 69,
 '}': 70,
 'UNK': 71}

In [119]:
# Ahora el texto se representa con una secuencia de caracteres
sequences = tk.texts_to_sequences(text['text'])
test_sequences = tk.texts_to_sequences(test_text['text'])

In [120]:
sequences[0]

[16,
 22,
 19,
 71,
 4,
 5,
 5,
 4,
 20,
 71,
 1,
 19,
 5,
 71,
 21,
 8,
 5,
 71,
 19,
 5,
 1,
 20,
 16,
 14,
 71,
 16,
 6,
 71,
 21,
 8,
 9,
 20,
 71,
 51,
 5,
 1,
 19,
 21,
 8,
 18,
 22,
 1,
 11,
 5,
 71,
 13,
 1,
 26,
 71,
 1,
 12,
 12,
 1,
 8,
 71,
 6,
 16,
 19,
 7,
 9,
 23,
 5,
 71,
 22,
 20,
 71,
 1,
 12,
 12]

In [121]:
test_sequences[0]

[10,
 22,
 20,
 21,
 71,
 8,
 1,
 17,
 17,
 5,
 14,
 5,
 4,
 71,
 1,
 71,
 21,
 5,
 19,
 19,
 9,
 2,
 12,
 5,
 71,
 3,
 1,
 19,
 71,
 3,
 19,
 1,
 20,
 8]

In [122]:
# Padding de cada secuencia para que todas tengan el mismo largo
data = pad_sequences(sequences, maxlen=1014, padding='post')
test_data = pad_sequences(test_sequences, maxlen=1014, padding='post')

In [123]:
data = np.array(data)
data.shape

(7434, 1014)

In [124]:
test_data = np.array(test_data)
test_data.shape

(3263, 1014)

In [125]:
train_classes = text['target'].values

#### CNN

In [126]:
size = len(tk.word_index)
size

71

In [127]:
embedding_weights = []
embedding_weights.append(np.zeros(size))

for char, i in tk.word_index.items():
    row = np.zeros(size)
    row[i-1] = 1
    embedding_weights.append(row)
    
embedding_weights = np.array(embedding_weights)

print(embedding_weights.shape)

(72, 71)


In [128]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

# Parametros
input_size = 1014
embedding_size = 71
fully_connected_layers = [1024, 1024]
num_of_classes = 1
dropout_p = 0.5
optimizer = 'adam'
loss = 'binary_crossentropy'

embedding_layer = Embedding(embedding_size+1, 
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

inputs = Input(shape=(input_size,), name='input', dtype='int64')

x = embedding_layer(inputs)

In [137]:
from keras.models import Sequential
from keras import layers

model = Sequential()

# Embedding
model.add(embedding_layer)

# Conv
model.add(layers.Conv1D(256, 3, activation='relu'))
model.add(layers.MaxPool1D(pool_size=4))

# fully connected
model.add(layers.Dense(1024, activation='relu'))

# Output
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_126"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_88 (Embedding)     (None, 1014, 71)          5112      
_________________________________________________________________
conv1d_144 (Conv1D)          (None, 1012, 256)         54784     
_________________________________________________________________
max_pooling1d_50 (MaxPooling (None, 253, 256)          0         
_________________________________________________________________
dense_186 (Dense)            (None, 253, 1024)         263168    
_________________________________________________________________
dense_187 (Dense)            (None, 253, 1)            1025      
Total params: 324,089
Trainable params: 324,089
Non-trainable params: 0
_________________________________________________________________


In [138]:
x_train, x_test, y_train, y_test = \
train_test_split(data, train_classes, test_size = 0.25, random_state = 123)

In [139]:
from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)
callbacks = [callback]

model1.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=10,
          verbose=2,
          callbacks=callback)

Epoch 1/10
44/44 - 27s - loss: 0.5642 - accuracy: 0.6738 - val_loss: 0.8166 - val_accuracy: 0.5912
Epoch 2/10
44/44 - 28s - loss: 0.5642 - accuracy: 0.6749 - val_loss: 0.8169 - val_accuracy: 0.5906
Epoch 00002: early stopping


In [102]:
def create_model(num_filters, kernel_size, vocab_size, embedding_dim, maxlen):
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen, weights=[embedding_weights], trainable=True))
    model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [104]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

param_grid = dict(num_filters=[32, 128, 144, 256],
                      kernel_size=[3, 5, 7],
                      vocab_size=[80],
                      embedding_dim=[79],
                      maxlen=[1014],
                      batch_size = [45,65,76,88])

model = KerasClassifier(build_fn=create_model,
                            epochs=15, validation_split=0.1,
                            verbose=1)

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=4, verbose=2, n_iter=5, n_jobs=1)

grid_result = grid.fit(x_train, y_train, callbacks=[callback])

# Evaluate testing set
#test_accuracy = grid.score(x_test, y_test)

# Save and evaluate results
s = ('Best Accuracy : {:.4f}\n{}\n\n\n')
output_string = s.format(
            grid_result.best_score_,
            grid_result.best_params_)
            
print(output_string)

Fitting 4 folds for each of 5 candidates, totalling 20 fits
[CV] vocab_size=80, num_filters=144, maxlen=1014, kernel_size=5, embedding_dim=79, batch_size=65 
Epoch 1/15


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


58/58 [==============================] - 8s 137ms/step - loss: 0.6478 - accuracy: 0.6196 - val_loss: 0.6309 - val_accuracy: 0.6277
Epoch 2/15
58/58 [==============================] - 8s 146ms/step - loss: 0.5804 - accuracy: 0.7140 - val_loss: 0.6026 - val_accuracy: 0.6921
Epoch 3/15
58/58 [==============================] - 9s 158ms/step - loss: 0.5202 - accuracy: 0.7565 - val_loss: 0.5745 - val_accuracy: 0.7017
Epoch 4/15
58/58 [==============================] - 10s 165ms/step - loss: 0.4567 - accuracy: 0.8089 - val_loss: 0.5597 - val_accuracy: 0.7208
Epoch 5/15
58/58 [==============================] - 10s 167ms/step - loss: 0.4013 - accuracy: 0.8408 - val_loss: 0.5499 - val_accuracy: 0.7232
Epoch 6/15
58/58 [==============================] - 10s 172ms/step - loss: 0.3492 - accuracy: 0.8703 - val_loss: 0.5300 - val_accuracy: 0.7184
Epoch 7/15
58/58 [==============================] - 10s 168ms/step - loss: 0.2939 - accuracy: 0.8987 - val_loss: 0.5165 - val_accuracy: 0.7589
Epoch 8/15
58

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s


58/58 [==============================] - 10s 168ms/step - loss: 0.6536 - accuracy: 0.6172 - val_loss: 0.6450 - val_accuracy: 0.6563
Epoch 2/15
58/58 [==============================] - 10s 171ms/step - loss: 0.5980 - accuracy: 0.7012 - val_loss: 0.6199 - val_accuracy: 0.6754
Epoch 3/15
58/58 [==============================] - 10s 168ms/step - loss: 0.5479 - accuracy: 0.7368 - val_loss: 0.6220 - val_accuracy: 0.6587
Epoch 00003: early stopping
22/22 [==============================] - 1s 48ms/step - loss: 0.5761 - accuracy: 0.7052
[CV]  vocab_size=80, num_filters=144, maxlen=1014, kernel_size=5, embedding_dim=79, batch_size=65, total=  31.7s
[CV] vocab_size=80, num_filters=144, maxlen=1014, kernel_size=5, embedding_dim=79, batch_size=65 
Epoch 1/15
58/58 [==============================] - 10s 173ms/step - loss: 0.6613 - accuracy: 0.5837 - val_loss: 0.6509 - val_accuracy: 0.6014
Epoch 2/15
58/58 [==============================] - 10s 172ms/step - loss: 0.5956 - accuracy: 0.7150 - val_loss:

Epoch 6/15
50/50 [==============================] - 4s 74ms/step - loss: 0.4975 - accuracy: 0.7765 - val_loss: 0.5697 - val_accuracy: 0.7064
Epoch 7/15
50/50 [==============================] - 4s 77ms/step - loss: 0.4599 - accuracy: 0.8036 - val_loss: 0.5663 - val_accuracy: 0.7017
Epoch 8/15
50/50 [==============================] - 4s 75ms/step - loss: 0.4259 - accuracy: 0.8206 - val_loss: 0.5742 - val_accuracy: 0.7136
Epoch 00008: early stopping
19/19 [==============================] - 0s 20ms/step - loss: 0.5332 - accuracy: 0.7487
[CV]  vocab_size=80, num_filters=32, maxlen=1014, kernel_size=5, embedding_dim=79, batch_size=76, total=  35.2s
[CV] vocab_size=80, num_filters=144, maxlen=1014, kernel_size=3, embedding_dim=79, batch_size=45 
Epoch 1/15
84/84 [==============================] - 8s 91ms/step - loss: 0.6735 - accuracy: 0.5731 - val_loss: 0.6603 - val_accuracy: 0.6253
Epoch 2/15
84/84 [==============================] - 8s 91ms/step - loss: 0.6323 - accuracy: 0.6576 - val_loss:

Epoch 1/15
43/43 [==============================] - 8s 175ms/step - loss: 0.6601 - accuracy: 0.5986 - val_loss: 0.6472 - val_accuracy: 0.6325
Epoch 2/15
43/43 [==============================] - 8s 175ms/step - loss: 0.6096 - accuracy: 0.6895 - val_loss: 0.6277 - val_accuracy: 0.6826
Epoch 3/15
43/43 [==============================] - 9s 199ms/step - loss: 0.5691 - accuracy: 0.7095 - val_loss: 0.6013 - val_accuracy: 0.6754
Epoch 4/15
43/43 [==============================] - 9s 200ms/step - loss: 0.5252 - accuracy: 0.7525 - val_loss: 0.5802 - val_accuracy: 0.7017
Epoch 5/15
43/43 [==============================] - 10s 225ms/step - loss: 0.4861 - accuracy: 0.7788 - val_loss: 0.5645 - val_accuracy: 0.7136
Epoch 6/15
43/43 [==============================] - 8s 187ms/step - loss: 0.4532 - accuracy: 0.7998 - val_loss: 0.5506 - val_accuracy: 0.6897
Epoch 7/15
43/43 [==============================] - 7s 172ms/step - loss: 0.4174 - accuracy: 0.8227 - val_loss: 0.5554 - val_accuracy: 0.7279
Epoch

Epoch 00006: early stopping
16/16 [==============================] - 1s 78ms/step - loss: 0.5702 - accuracy: 0.6899
[CV]  vocab_size=80, num_filters=128, maxlen=1014, kernel_size=7, embedding_dim=79, batch_size=88, total= 1.4min
Epoch 1/15


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 20.7min finished


58/58 [==============================] - 15s 259ms/step - loss: 0.6436 - accuracy: 0.6324 - val_loss: 0.6393 - val_accuracy: 0.6165
Epoch 2/15
58/58 [==============================] - 15s 263ms/step - loss: 0.5854 - accuracy: 0.7070 - val_loss: 0.6069 - val_accuracy: 0.6685
Epoch 3/15
58/58 [==============================] - 15s 262ms/step - loss: 0.5201 - accuracy: 0.7622 - val_loss: 0.5681 - val_accuracy: 0.7204
Epoch 4/15
58/58 [==============================] - 15s 261ms/step - loss: 0.4545 - accuracy: 0.8116 - val_loss: 0.5856 - val_accuracy: 0.7007
Epoch 00004: early stopping
Best Accuracy : 0.7530
{'vocab_size': 80, 'num_filters': 128, 'maxlen': 1014, 'kernel_size': 7, 'embedding_dim': 79, 'batch_size': 88}





In [105]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

param_grid = dict(num_filters=[128, 144, 256],
                      kernel_size=[3, 5, 7],
                      vocab_size=[80],
                      embedding_dim=[79],
                      maxlen=[1014],
                      batch_size = [45,65,76,88])

model = KerasClassifier(build_fn=create_model,
                            epochs=15, validation_split=0.1,
                            verbose=1)

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=4, verbose=2, n_iter=5, n_jobs=1)

grid_result = grid.fit(x_train, y_train, callbacks=[callback])

# Evaluate testing set
#test_accuracy = grid.score(x_test, y_test)

# Save and evaluate results
s = ('Best Accuracy : {:.4f}\n{}\n\n\n')
output_string = s.format(
            grid_result.best_score_,
            grid_result.best_params_)
            
print(output_string)

Fitting 4 folds for each of 5 candidates, totalling 20 fits
[CV] vocab_size=80, num_filters=128, maxlen=1014, kernel_size=7, embedding_dim=79, batch_size=65 
Epoch 1/15


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


58/58 [==============================] - 9s 162ms/step - loss: 0.6624 - accuracy: 0.6119 - val_loss: 0.6445 - val_accuracy: 0.6635
Epoch 2/15
58/58 [==============================] - 10s 180ms/step - loss: 0.5977 - accuracy: 0.7023 - val_loss: 0.6099 - val_accuracy: 0.6492
Epoch 3/15
58/58 [==============================] - 13s 221ms/step - loss: 0.5454 - accuracy: 0.7392 - val_loss: 0.5790 - val_accuracy: 0.6826
Epoch 4/15
58/58 [==============================] - 12s 199ms/step - loss: 0.4762 - accuracy: 0.7972 - val_loss: 0.5567 - val_accuracy: 0.7160
Epoch 5/15
58/58 [==============================] - 14s 247ms/step - loss: 0.4166 - accuracy: 0.8307 - val_loss: 0.5500 - val_accuracy: 0.7208
Epoch 6/15
58/58 [==============================] - 14s 237ms/step - loss: 0.3518 - accuracy: 0.8708 - val_loss: 0.5452 - val_accuracy: 0.7470
Epoch 7/15
58/58 [==============================] - 14s 244ms/step - loss: 0.2987 - accuracy: 0.8955 - val_loss: 0.5580 - val_accuracy: 0.7422
Epoch 00007

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s


58/58 [==============================] - 12s 207ms/step - loss: 0.6520 - accuracy: 0.6297 - val_loss: 0.6426 - val_accuracy: 0.6778
Epoch 2/15
58/58 [==============================] - 12s 213ms/step - loss: 0.5913 - accuracy: 0.7169 - val_loss: 0.6070 - val_accuracy: 0.6826
Epoch 3/15
58/58 [==============================] - 12s 199ms/step - loss: 0.5167 - accuracy: 0.7658 - val_loss: 0.5656 - val_accuracy: 0.7088
Epoch 4/15
58/58 [==============================] - 12s 214ms/step - loss: 0.4473 - accuracy: 0.8142 - val_loss: 0.5438 - val_accuracy: 0.7446
Epoch 5/15
58/58 [==============================] - 12s 212ms/step - loss: 0.3793 - accuracy: 0.8562 - val_loss: 0.5298 - val_accuracy: 0.7518
Epoch 6/15
58/58 [==============================] - 12s 212ms/step - loss: 0.3213 - accuracy: 0.8830 - val_loss: 0.5231 - val_accuracy: 0.7470
Epoch 7/15
58/58 [==============================] - 11s 189ms/step - loss: 0.2635 - accuracy: 0.9144 - val_loss: 0.5554 - val_accuracy: 0.7112
Epoch 0000

Epoch 1/15
50/50 [==============================] - 9s 170ms/step - loss: 0.6604 - accuracy: 0.5969 - val_loss: 0.6250 - val_accuracy: 0.6468
Epoch 2/15
50/50 [==============================] - 9s 171ms/step - loss: 0.5989 - accuracy: 0.7029 - val_loss: 0.5957 - val_accuracy: 0.6754
Epoch 3/15
50/50 [==============================] - 9s 173ms/step - loss: 0.5426 - accuracy: 0.7454 - val_loss: 0.5990 - val_accuracy: 0.6611
Epoch 00003: early stopping
19/19 [==============================] - 1s 47ms/step - loss: 0.5968 - accuracy: 0.6655
[CV]  vocab_size=80, num_filters=128, maxlen=1014, kernel_size=5, embedding_dim=79, batch_size=76, total=  27.9s
[CV] vocab_size=80, num_filters=144, maxlen=1014, kernel_size=7, embedding_dim=79, batch_size=45 
Epoch 1/15
84/84 [==============================] - 12s 147ms/step - loss: 0.6415 - accuracy: 0.6257 - val_loss: 0.6228 - val_accuracy: 0.6539
Epoch 2/15
84/84 [==============================] - 12s 144ms/step - loss: 0.5516 - accuracy: 0.7337 - v

84/84 [==============================] - 26s 309ms/step - loss: 0.6296 - accuracy: 0.6551 - val_loss: 0.6199 - val_accuracy: 0.6516
Epoch 2/15
84/84 [==============================] - 38s 455ms/step - loss: 0.5252 - accuracy: 0.7582 - val_loss: 0.5635 - val_accuracy: 0.7112
Epoch 3/15
84/84 [==============================] - 3168s 38s/step - loss: 0.4296 - accuracy: 0.8225 - val_loss: 0.5414 - val_accuracy: 0.7351
Epoch 4/15
84/84 [==============================] - 36s 433ms/step - loss: 0.3444 - accuracy: 0.8754 - val_loss: 0.5558 - val_accuracy: 0.7399
Epoch 00004: early stopping
31/31 [==============================] - 4s 115ms/step - loss: 0.4996 - accuracy: 0.7703
[CV]  vocab_size=80, num_filters=128, maxlen=1014, kernel_size=7, embedding_dim=79, batch_size=45, total=54.6min
[CV] vocab_size=80, num_filters=144, maxlen=1014, kernel_size=3, embedding_dim=79, batch_size=88 
Epoch 1/15
43/43 [==============================] - 31s 718ms/step - loss: 0.6639 - accuracy: 0.5957 - val_loss

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 653.6min finished


112/112 [==============================] - 16s 141ms/step - loss: 0.6544 - accuracy: 0.6024 - val_loss: 0.6679 - val_accuracy: 0.5914
Epoch 2/15
112/112 [==============================] - 15s 131ms/step - loss: 0.5813 - accuracy: 0.6909 - val_loss: 0.5941 - val_accuracy: 0.6738
Epoch 3/15
112/112 [==============================] - 15s 131ms/step - loss: 0.4976 - accuracy: 0.7768 - val_loss: 0.5635 - val_accuracy: 0.7097
Epoch 4/15
112/112 [==============================] - 14s 129ms/step - loss: 0.4317 - accuracy: 0.8126 - val_loss: 0.5411 - val_accuracy: 0.6989
Epoch 5/15
112/112 [==============================] - 15s 130ms/step - loss: 0.3631 - accuracy: 0.8545 - val_loss: 0.5171 - val_accuracy: 0.7509
Epoch 6/15
112/112 [==============================] - 14s 129ms/step - loss: 0.3016 - accuracy: 0.8914 - val_loss: 0.5139 - val_accuracy: 0.7599
Epoch 7/15
112/112 [==============================] - 15s 132ms/step - loss: 0.2385 - accuracy: 0.9266 - val_loss: 0.5233 - val_accuracy: 0.7

In [106]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

param_grid = dict(num_filters=[256],
                      kernel_size=[3, 5, 7],
                      vocab_size=[80],
                      embedding_dim=[79],
                      maxlen=[1014],
                      batch_size = [45,65,76,88])

model = KerasClassifier(build_fn=create_model,
                            epochs=15, validation_split=0.1,
                            verbose=1)

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=4, verbose=2, n_iter=5, n_jobs=1)

grid_result = grid.fit(x_train, y_train, callbacks=[callback])

# Evaluate testing set
#test_accuracy = grid.score(x_test, y_test)

# Save and evaluate results
s = ('Best Accuracy : {:.4f}\n{}\n\n\n')
output_string = s.format(
            grid_result.best_score_,
            grid_result.best_params_)
            
print(output_string)

Fitting 4 folds for each of 5 candidates, totalling 20 fits
[CV] vocab_size=80, num_filters=256, maxlen=1014, kernel_size=5, embedding_dim=79, batch_size=76 
Epoch 1/15


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


50/50 [==============================] - 15s 294ms/step - loss: 0.6610 - accuracy: 0.5848 - val_loss: 0.6437 - val_accuracy: 0.6444
Epoch 2/15
50/50 [==============================] - 16s 319ms/step - loss: 0.5959 - accuracy: 0.7097 - val_loss: 0.6082 - val_accuracy: 0.6492
Epoch 3/15
50/50 [==============================] - 15s 308ms/step - loss: 0.5356 - accuracy: 0.7528 - val_loss: 0.5990 - val_accuracy: 0.6921
Epoch 4/15
50/50 [==============================] - 19s 379ms/step - loss: 0.4670 - accuracy: 0.8025 - val_loss: 0.5634 - val_accuracy: 0.6993
Epoch 5/15
50/50 [==============================] - 21s 417ms/step - loss: 0.4066 - accuracy: 0.8405 - val_loss: 0.5504 - val_accuracy: 0.7041
Epoch 6/15
50/50 [==============================] - 16s 320ms/step - loss: 0.3599 - accuracy: 0.8604 - val_loss: 0.5349 - val_accuracy: 0.7494
Epoch 7/15
50/50 [==============================] - 16s 322ms/step - loss: 0.3018 - accuracy: 0.8953 - val_loss: 0.5246 - val_accuracy: 0.7399
Epoch 8/15

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.3min remaining:    0.0s


50/50 [==============================] - 16s 316ms/step - loss: 0.6487 - accuracy: 0.6308 - val_loss: 0.6447 - val_accuracy: 0.6539
Epoch 2/15
50/50 [==============================] - 16s 319ms/step - loss: 0.5849 - accuracy: 0.7033 - val_loss: 0.6031 - val_accuracy: 0.6659
Epoch 3/15
50/50 [==============================] - 16s 315ms/step - loss: 0.5220 - accuracy: 0.7608 - val_loss: 0.5640 - val_accuracy: 0.7017
Epoch 4/15
50/50 [==============================] - 16s 314ms/step - loss: 0.4519 - accuracy: 0.8062 - val_loss: 0.5342 - val_accuracy: 0.7399
Epoch 5/15
50/50 [==============================] - 16s 316ms/step - loss: 0.3905 - accuracy: 0.8418 - val_loss: 0.5187 - val_accuracy: 0.7542
Epoch 6/15
50/50 [==============================] - 16s 313ms/step - loss: 0.3238 - accuracy: 0.8844 - val_loss: 0.5085 - val_accuracy: 0.7709
Epoch 7/15
50/50 [==============================] - 16s 312ms/step - loss: 0.2610 - accuracy: 0.9147 - val_loss: 0.5159 - val_accuracy: 0.7780
Epoch 0000

Epoch 00008: early stopping
22/22 [==============================] - 2s 87ms/step - loss: 0.5755 - accuracy: 0.7554
[CV]  vocab_size=80, num_filters=256, maxlen=1014, kernel_size=5, embedding_dim=79, batch_size=65, total= 2.3min
[CV] vocab_size=80, num_filters=256, maxlen=1014, kernel_size=5, embedding_dim=79, batch_size=65 
Epoch 1/15
58/58 [==============================] - 16s 279ms/step - loss: 0.6412 - accuracy: 0.6373 - val_loss: 0.6520 - val_accuracy: 0.6277
Epoch 2/15
58/58 [==============================] - 16s 284ms/step - loss: 0.5761 - accuracy: 0.7079 - val_loss: 0.5805 - val_accuracy: 0.6826
Epoch 3/15
58/58 [==============================] - 16s 283ms/step - loss: 0.5012 - accuracy: 0.7776 - val_loss: 0.5462 - val_accuracy: 0.7279
Epoch 4/15
58/58 [==============================] - 16s 282ms/step - loss: 0.4334 - accuracy: 0.8198 - val_loss: 0.5546 - val_accuracy: 0.7327
Epoch 00004: early stopping
22/22 [==============================] - 2s 86ms/step - loss: 0.5223 - ac

50/50 [==============================] - 20s 400ms/step - loss: 0.3874 - accuracy: 0.8554 - val_loss: 0.5915 - val_accuracy: 0.7327
Epoch 00004: early stopping
19/19 [==============================] - 2s 110ms/step - loss: 0.5543 - accuracy: 0.7373
[CV]  vocab_size=80, num_filters=256, maxlen=1014, kernel_size=7, embedding_dim=79, batch_size=76, total= 1.4min
Epoch 1/15


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 41.6min finished


78/78 [==============================] - 23s 293ms/step - loss: 0.6345 - accuracy: 0.6466 - val_loss: 0.6302 - val_accuracy: 0.6703
Epoch 2/15
78/78 [==============================] - 24s 306ms/step - loss: 0.5434 - accuracy: 0.7419 - val_loss: 0.5671 - val_accuracy: 0.7115
Epoch 3/15
78/78 [==============================] - 23s 301ms/step - loss: 0.4601 - accuracy: 0.7993 - val_loss: 0.5349 - val_accuracy: 0.7348
Epoch 4/15
78/78 [==============================] - 23s 289ms/step - loss: 0.3972 - accuracy: 0.8312 - val_loss: 0.5229 - val_accuracy: 0.7599
Epoch 5/15
78/78 [==============================] - 22s 278ms/step - loss: 0.3380 - accuracy: 0.8704 - val_loss: 0.5034 - val_accuracy: 0.7652
Epoch 6/15
78/78 [==============================] - 22s 282ms/step - loss: 0.2767 - accuracy: 0.9047 - val_loss: 0.4966 - val_accuracy: 0.7742
Epoch 7/15
78/78 [==============================] - 22s 280ms/step - loss: 0.2285 - accuracy: 0.9257 - val_loss: 0.5065 - val_accuracy: 0.7688
Epoch 0000